<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Teradataml Python Basics
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<table>
    <tr>
        <td width = '50%' style = 'vertical-align:top;font-size:16px;font-family:Arial'><br><p>Teradataml is a library that allows python programmers and developers to access the power of Vantage.</p>
            <p>Developers can use common data management functions and methods based on Python Pandas.  Teradataml translates python directives to underlying SQL in order to process directly on the Vantage system without costly data movement.</p>
        <p>Additionally, teradataml provides simple functions providing access to machine learning, open analytics frameworks, and powerful advanced analytical capabilities.</td>
        <td width = '50%'><img src = 'images/Functional_Diagram.png'></td>
    </tr>
    </table>




<p style = 'font-size:16px;font-family:Arial'>This notebook will cover the very basics of the Teradataml package and is a technical demonstration of different functionalities of Teradataml. This is not a business outcome type demo.  Please see the Getting Started Guide online <a href = 'https://docs.teradata.com/search/all?query=Teradata%25C2%25AE+Python+Package+User+Guide&content-lang=en-US'>here</a></p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Contents</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
<li>Connecting to Vantage
    <ul>
        <li>Initiate a connection to Vantage</li>
    </ul>
</li>    
<li>Teradataml Basics
    <ul>
        <li>Create a Teradata DataFrame (virtual DataFrame)</li>
        <li>Aggregations</li>
        <li>Transformations</li>
        <li>SQL Functions</li>
        <li>Joins</li>
        <li>Bring the data to the client - Pandas</li>
    </ul>
    <li>Cleanup</li>
</ol>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Install and import the necessary libraries</b></p>

<p style = 'font-size:16px;font-family:Arial'>Python is a programming language that allows developers to add new functionalities by using libraries. These libraries contain pre-written code contributed by various individuals and companies. To use these libraries, we use the "pip" command to install them. When we install a library, it may also require other libraries to be installed along with it.
<br>
<br>
To make things easier for users and to save time during startup, we have already pre-installed and tested the libraries used in the ClearScape Analytics Experience Jupyter notebooks with Python. The pip commands for installing these libraries are commented out, which means they are not executed by default. You can uncomment these commands if you want to install newer versions of the libraries or if you encounter any issues and need to reinstall them.</p>

<p style = 'font-size:16px;font-family:Arial'>Upgrade to latest version of Teradataml before importing it.</p>

<p style = 'font-size:16px;font-family:Arial'>Note:<b> %%capture</b> suppresses the display of installation steps of the following packages</p>

In [ ]:
# %%capture
# !pip install --upgrade teradataml --user

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
# getpass to ask password to user and prevent storing it plain in the Notebook
import getpass
import pandas as pd

# import all Teradataml functions and supporting libraries
from teradataml import *
from teradataml.table_operators.Script import Script
from sqlalchemy import func

display.max_rows = 5

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=teradataml_Python_Basics_Python.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.

In [ ]:
# %run -i ../../UseCases/run_procedure.py "call get_data('DEMO_DataScienceExploration_cloud');"   # Takes about 1 minute 
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_DataScienceExploration_local');"  # Takes about 2 minutes

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Create a Teradata DataFrame (virtual DataFrame)</b>
<p style = 'font-size:16px;font-family:Arial'>The teradataml DataFrame module function can create a pointer to a table or a SQL statement in the target Vantage system.  Note, no data is copied back to the client, and all operations on the data are translated to SQL and executed in Vantage.</p>

In [ ]:
tdf = DataFrame(in_schema('DEMO_DataScienceExploration', 'House_Prices'))

<p style = 'font-size:16px;font-family:Arial'>Extract a few rows - note, only the rows needed to satisfy the head() method are returned to the client</p>

In [ ]:
tdf.head(n = 5)

<p style = 'font-size:16px;font-family:Arial'>Look at the underlying query by using the show_query() method.</p>

In [ ]:
tdf.head().show_query()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Aggregations</b>
<p style = 'font-size:16px;font-family:Arial'>Various aggregations are available for grouping, windowing, time series, etc.</p>

<p style = 'font-size:16px;font-family:Arial'>Select only the required Columns in the Dataframe.</p>

In [ ]:
tdf_group =  tdf.select(["bedrooms", "price"])

In [ ]:
# simple groupby:
tdf_group.groupby('bedrooms').sum().sort("bedrooms")

<p style = 'font-size:16px;font-family:Arial'>The above output shows the sum of the price grouped by the number of bedrooms.</p>

<p style = 'font-size:16px;font-family:Arial'>Multiple aggregate function can be combined using the agg() method as shown below.</p>

In [ ]:
tdf_count = tdf.select(["bedrooms", "id","bathrooms"])

In [ ]:
# Groupby using the agg() method
#  Valid aggregation  values are 'count', 'sum', 'min', 'max', 'mean', 'std', 'percentile', 'unique','median', 'var'
tdf_count.groupby('bedrooms').agg(['count','min']).sort('bedrooms')

<p style = 'font-size:16px;font-family:Arial'>The output shows the grouping by bedrooms. It showcases the count of all the properties with the specific number of bathrooms and the count and minimum of bathrooms</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Transformations</b>
<p style = 'font-size:16px;font-family:Arial'>Assign method can be used to create new columns as the result of an expression.</p>

In [ ]:
# As with Pandas, the method call returns an object of teradataml DataFrame
# Use self-assignment to create the column in the existing dataframe if desired:

tdf.assign(price_per_bed = tdf['price'] / tdf['bedrooms'])

<p style = 'font-size:16px;font-family:Arial'>Please scroll to the right most column to check the calculated column.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>SQL Functions</b>
<p style = 'font-size:16px;font-family:Arial'>teradataml supports the following categories of SQL functions with SQLAlchemy extension:</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Aggregate Functions :-  Avg, Corr, Count, Covar_pop, Covar_samp etc. (total 23 functions)</li>
    <li>Arithmetic, Hyperbolic and Trigonometric Functions :- ABS, CASE_N, CEIL, DEGREES, RADIANS etc. (total 32 functions)</li>
<li>Bit Byte Manipulation Functions :- BITAND/OR/NOT/XOR, COUNTSET, GETBIT, ROTATELEFT, SETBIT etc (total 13 functions)</li>
<li>Built-In Functions :- CURRENT_DATE, CURRENT_TIME, CURRENT_TIMESTAMP</li>
<li>Hash Related Functions :- HASHAMP, HASHBAKAMP, HASHBUCKET, HASHROW</li>
<li>Regular Expression Functions :- REGEXP_SUBSTR, REGEXP_REPLACE, REGEXP_INSTR, REGEXP_SIMILAR</li>
<li>String Functions :- ASCII, CHAR2HEXINT, CHR, CONCAT, EDITDISTANCE etc (total 27 functions)</li>
<li>Window Aggregate Functions :- CSUM, CUME_DIST, DENSE_RANK, FIRST_VALUE, LAST_VALUE etc.(total 18 functions)</li>
</ul>

<p style = 'font-size:16px;font-family:Arial'><a href = 'https://docs.teradata.com/r/Teradata-Package-for-Python-User-Guide/November-2021/teradataml-Extension-with-SQLAlchemy/Accessing-Vantage-SQL-Functions/Supported-SQL-Functions' >SQL Functions</a></p>


In [ ]:
# Pearson Correlation Coefficient - in this example, for our data set,
# what's the correlation between price and square footage?

corr_func = func.corr(tdf['price'].expression, tdf['sqft_living'].expression)


#Setting drop_columns = True here
df_corr = tdf.assign(drop_columns = True, corr_ = corr_func)

print(df_corr)

<p style = 'font-size:16px;font-family:Arial'>Returns the Sample Pearson product moment correlation coefficient of its arguments for all non-null data point pairs. Here we are finding the correlation coefficient between price and square footage. A correlation of 1 or +1 shows a perfect positive correlation, which means both the variables move in the same direction. A correlation of -1 shows a perfect negative correlation, which means as one variable goes down, the other goes up. The value of 0.7 is really good, means that the relativity between price and sqft living is good. </p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Joins</b>
<p style = 'font-size:16px;font-family:Arial'>Pandas-Style joins can be used to create SQL joins in Vantage</p>

In [ ]:
# Create a new dataframe using a SQL statement

qry = '''
SELECT id,
CASE 
    WHEN waterfront = 0 THEN 'no'
    ELSE 'yes'
END str_waterfront
FROM "DEMO_DataScienceExploration"."House_Prices"
'''
tdf_waterfront = DataFrame.from_query(qry)
tdf_waterfront.head()

<p style = 'font-size:16px;font-family:Arial'>The original dataframe (tdf) and the Dataframe created using the query above(tdf_waterfront) will be joined using the 'id' column. tdf_joined_data is the resultant dataframe.</p>

In [ ]:
#join these two dataframes together

tdf_joined_data = tdf.join(tdf_waterfront, on = 'id', how = 'left', rsuffix = 'r', lsuffix = 'l')
tdf_joined_data

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Bring the data to the client - Pandas</b>
<p style = 'font-size:16px;font-family:Arial'>The to_pandas() function creates a pandas DataFrame from a teradataml DataFrame. This method will selectively retrieve data to the client</p>

In [ ]:
df = tdf_joined_data.to_pandas(all_rows = True)

In [ ]:
tdf_joined_data.head(n = 5)

<p style = 'font-size:16px;font-family:Arial'>Multiple aggregate function can be combined using the agg() method as shown below. In the pandas dataframe the columns to be used along with the aggregation can also be specified.</p>

In [ ]:
tdf_joined_data.groupby('bedrooms').agg({'id_l': 'count', 'price': 'sum'}).sort('bedrooms')

<p style = 'font-size:16px;font-family:Arial'>Data from the dataframe can also be used to plot graphs as required.</p>

In [ ]:
df['price'].plot(kind = 'hist', bins = 10)

<p style = 'font-size:16px;font-family:Arial'>The above graph is a histogram which is distributing the entire data into 10 bins and the Y-axis is the frequency of prices of the properties. In the data most of the properties(70k) fall in the first bin and 8k in the 2nd bin and so on. The hist function takes a number of arguments, the key one being the bins argument, which specifies the number of equal-width bins in the range.</p>
<p style = 'font-size:16px;font-family:Arial'>A histogram is basically used to represent data provided in a form of some groups. It is accurate method for the graphical representation of numerical data distribution. It is a type of bar plot where X-axis represents the bin ranges while Y-axis gives information about frequency.</p>


<p style = 'font-size:16px;font-family:Arial'>As specified earlier this notebook is just a functional representation of the teradataml library functions and a sample display of the usage for these functions. For other functions and details please refer to the Getting Started Guide online <a href = 'https://docs.teradata.com/search/all?query=Teradata%25C2%25AE+Python+Package+User+Guide&content-lang=en-US'>here</a></p></p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Cleanup</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_DataScienceExploration');"  # optional if you want to access the data later
#Takes 5 seconds

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'>It is a good practice to remove the context that we created to connect to Vantage. The remove_context function removes the current context associated with the Vantage connection.remove_context() not only closes the connection but also garbage collects the intermediate views and tables created by teradataml. Teradata recommends calling remove_context() to end a session, so that intermediate views and tables created by teradataml are garbage collected.</p>

In [ ]:
remove_context()

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>